# **Example 04: Unrolled network on complex-valued data with complex-valued operations**
We first define the data pipelines to feed the data into training, validation and test set. The MNIST database is used for showcasing. Since MNIST are real-valued images, a phase is simulated and added to the images to generate a complex-valued input. The fourier transformed image serves as k-space for data consistency. Single-MR-coil processing is performed. A retrospective undersampling according to a Parallel Imaging (PI) or Compressed Sensing (CS) like trajectory is simulated and applied to the data. The task of the network is to reconstruct the undersampled data with complex-valued operations.

To enable GPU support in Google Colab, please go to `Edit -> Notebook settings` and select `GPU` as hardware accelerator.

In [ ]:
# inspect the available GPU hardware
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!nvidia-smi

## Database pipeline
Here starts the main part of the script. First define the data pipelines (in the form of generator functions) for training, validation and test set. Retrospective undersampling is performed inside the generator functions.

In [ ]:
import tensorflow as tf
import numpy as np
import tutorial
import merlintf

# initialize some parameters
accelerations = [2, 4]  # simulate retrospectively accelerations in the range of e.g. 2x to 4x
accel_type = 'PI'  # simulated undersampling strategy: 'PI' = Parallel Imaging, 'CS' = Compressed Sensing
center = 0.1  # percent of fully sampled central region along ky phase-encoding, e.g. 0.1 := floor(10% * 28) ky center lines = 2 ky center lines 

# Data Generators (Data pipeline) for complex-valued data with k-space
# training set
training_generator = tutorial.datasets.ComplexRawDataGeneratorMNIST(batch_size=32, 
                                    accelerations=accelerations,
                                    accel_type=accel_type,
                                    center=center, 
                                    shuffle=True,
                                    mode='train')

# validation set
validation_generator = tutorial.datasets.ComplexRawDataGeneratorMNIST(batch_size=32, 
                                    accelerations=accelerations,
                                    accel_type=accel_type,
                                    center=center,
                                    shuffle=False,
                                    mode='val')

# test set
# ideally testing should be performed on prospectively accelerated acquisitions
test_generator = tutorial.datasets.ComplexRawDataGeneratorMNIST(batch_size=1, 
                                    accelerations=accelerations,
                                    accel_type=accel_type,
                                    center=center,
                                    shuffle=False,
                                    mode='test')

print('Training batches to process:', len(training_generator))
print('Validation batches to process:', len(validation_generator))
print('Test samples to process:', len(test_generator))

## Model
Define the CNN model as an unrolled network with intermittant data consistency blocks.

### 3-layer (residual) convolutional neural network (CNN) denoising regularizer

In [ ]:
# Define denoiser model
class Denoiser(tf.keras.Model):
    # initialize the required layers
    def __init__(self, filters, kernel_size=(3,3), strides=(1,1), use_bias=True, activation=None, is_residual=False, name='denoiser'):
        super().__init__(name=name)
        self.is_residual = is_residual
        self.conv_out1 = merlintf.keras.layers.ComplexConv2D(filters=filters,
                                kernel_size=kernel_size,
                                strides=strides,
                                padding='SAME',
                                use_bias=use_bias,
                                activation=activation)

        self.conv_out2 = merlintf.keras.layers.ComplexConv2D(filters=filters,
                                kernel_size=kernel_size,
                                strides=strides,
                                padding='SAME',
                                use_bias=use_bias,
                                activation=activation)

        self.conv_out3 = merlintf.keras.layers.ComplexConv2D(filters=1,
                                kernel_size=kernel_size,
                                strides=strides,
                                padding='SAME',
                                use_bias=True,
                                activation=None)
        
    def call(self, inputs):
        x = self.conv_out1(inputs)
        x = self.conv_out2(x)
        x = self.conv_out3(x)
        if self.is_residual:
            x = tf.keras.layers.Add()([inputs, x])
        return x


### Unrolled reconstruction network
Combine denoising regularizer and data consistency blocks to an unrolled reconstruction network

In [ ]:
# Define unrolled reconstruction model
class UnrolledNetwork(tf.keras.Model):
    # initialize the required layers
    def __init__(self, cascades, shared_params=True, is_residual=True, activation='ModReLU', name=None):
        super().__init__(name=name)

        self.T = 1 if shared_params else cascades  # shared denoiser network or (new) cascaded denoisers
        self.T_end = cascades  # number of cascades

        self.denoiser = [Denoiser(filters=8, is_residual=is_residual, activation=activation) for _ in range(self.T)]  # prepare denoising networks

        A = merlintf.keras.layers.ForwardOp(center=True)
        AH = merlintf.keras.layers.AdjointOp(center=True)

        self.dc = [merlintf.keras.layers.DCGD(A, AH, weight_init=1.0, max_iter=10) for _ in range(self.T)]  # prepare data consistency blocks

    # build the model in the forward path
    def call(self, inputs):
        x = inputs[0]                                       # undersampled image
        for i in range(self.T_end):                         # unrolled network
            ii = i % self.T
            x = self.denoiser[ii](x)                    # denoising regularizer
            x = self.dc[ii]([x,] + list(inputs[1:]))    # data consistency
        return x
    
# instantiate a keras model as sub-class model
# define network with model subclassing to control forward/backward path, training procedure, etc.
model = UnrolledNetwork(cascades=5)
# make one forward pass to initalize model
inputs, _ = training_generator.__getitem__(0)
output = model(inputs)

# print model overview
print(model.summary())

### Build model
Compile the model, assign an optimizer, loss function and validation metrics. Prepare some keras callbacks to monitor training progress.

In [ ]:
# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),      # used optimizer with chosen learning rate
              loss='mse',                                                   # loss function 
              metrics=['mse', 'mae'])                                       # evaluation metrics (for training and validation set)

# define callbacks to monitor model
keras_callbacks = tutorial.get_callbacks(validation_generator, model)

## Tensorboard
Start the Tensorboard [optional] to monitor training progress and display validation outputs.

In [ ]:
# start Tensorboard
%load_ext tensorboard
%tensorboard --logdir=logs

## Training
Train the configured and compiled model. Monitor training progress with validation set.

In [ ]:
# train model with training set and evaluate its performance with the validation set
model.fit(training_generator,                       # training set
          validation_data=validation_generator,     # validation set
          epochs=3,                                 # number of epochs to train the model
          callbacks=keras_callbacks)                # callbacks to monitor or control training

## Testing
Test the trained model to predict a denoised output and to display performance (metrics) on test set.

In [ ]:
# evaluate trained model
loss_metric_test = model.evaluate(test_generator)

In [ ]:
# predict with trained model
inputs, outputs = test_generator.__getitem__(0)
predicted_output = model.predict(inputs)

# display the predicted output
import matplotlib.pyplot as plt
icase = 0  # display the first example
plt.figure()
plt.subplot(2,3,1)
plt.imshow(np.squeeze(np.abs(inputs[0][icase,])), cmap='gray')
plt.title('Magnitude - Noisy')
plt.axis('off')
plt.subplot(2,3,2)
plt.imshow(np.squeeze(np.abs(predicted_output[icase,])), cmap='gray')
plt.title('Magnitude - Recon')
plt.axis('off')
plt.subplot(2,3,3)
plt.imshow(np.squeeze(np.abs(outputs[icase,])), cmap='gray')
plt.title('Magnitude - Target')
plt.axis('off')
plt.subplot(2,3,4)
plt.imshow(np.squeeze(np.angle(inputs[0][icase,])), vmin=-np.pi, vmax=np.pi)
plt.title('Phase - Noisy')
plt.axis('off')
plt.subplot(2,3,5)
plt.imshow(np.squeeze(np.angle(predicted_output[icase,])), vmin=-np.pi, vmax=np.pi)
plt.title('Phase - Recon')
plt.axis('off')
plt.subplot(2,3,6)
plt.imshow(np.squeeze(np.angle(outputs[icase,])), vmin=-np.pi, vmax=np.pi)
plt.title('Phase - Target')
plt.axis('off')
plt.show()